# Converting image file information to CSV
#### 데이터셋 형식 변경 : .jpg파일 CSV파일로 변환
(AiHub Korean Emotion Dataset Conversion Manual 참고)

In [1]:
import csv
import os
import dlib
import numpy as np
from PIL import Image

csv_filename = "data1.csv"  # !!! 저장할 csv 파일 이름 정하기 !!!
fieldnames = ["dataset", "filename", "subject", "expression", "bbox", "mark", "pose"]

# CSV 파일 생성
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(fieldnames)

    # 파일 경로 탐색
    image_path = "C:/data/angry30/"  # !!! 중요 : 경로수정필요 !!!
    image_files = os.listdir(image_path)

    # 얼굴 검출기 초기화
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat") 
    # !!! 얼굴 랜드마크 예측 모델(shape_predictor_68_face_landmarks.dat) !!!

    for filename in image_files:
        # 이미지 파일 경로
        filepath = os.path.join(image_path, filename)

        # 이미지 파일 로드
        image = Image.open(filepath)
        image_array = np.array(image)

        # 얼굴 검출
        dets = detector(image_array, 1)

        if len(dets) > 0:
            # 첫 번째 얼굴에 대한 랜드마크 추출
            shape = predictor(image_array, dets[0])

            # bbox 추출 (top, left, width, height)
            bbox = f"{dets[0].top()};{dets[0].left()};{dets[0].width()};{dets[0].height()}"

            # mark 추출 (EyeR, EyeL, Nose, En R)
            mark = ""
            for i in [36, 39, 30, 42]:  # 랜드마크 인덱스
                x = shape.part(i).x
                y = shape.part(i).y
                mark += f"{x};{y};"
            mark = mark.rstrip(";")

            # CSV 파일에 데이터 작성
            dataset = "affectnet"
            subject = ""
            expression = "angry"
            pose = "0.0:0.0:0.0"
            writer.writerow([dataset, filepath, subject, expression, bbox, mark, pose])
        else:
            # 얼굴이 검출되지 않은 경우
            dataset = "affectnet"
            subject = ""
            expression = "no_face"
            bbox = ""
            mark = ""
            pose = "0.0:0.0:0.0"
            writer.writerow([dataset, filepath, subject, expression, bbox, mark, pose])
